자연어 처리(NLP)에서 데이터 준비와 모델링을 위한 전반적인 순서를 다음과 같이 요약할 수 있습니다. 이 과정은 문제의 유형이나 데이터의 특성에 따라 다소 달라질 수 있지만, 일반적으로 다음의 단계로 구성됩니다.

[데이터 수집] → [데이터 전처리] → [특징 추출] → [모델 학습] → [모델 평가] → [하이퍼파라미터 튜닝] → [모델 배포] → [모델 모니터링]

1. 데이터 수집 (Data Collection)
웹 스크래핑, API, 파일 등 다양한 방법으로 원시 텍스트 데이터를 수집합니다.
2. 데이터 전처리 (Data Preprocessing)
수집한 데이터를 정제하여 모델 학습에 적합한 형식으로 변환합니다. 이 과정에는 다음의 단계가 포함됩니다:
인코딩: 올바른 문자 인코딩을 적용합니다 (예: UTF-8).
토큰화: 문장을 단어 또는 문장 단위로 분리합니다.
정제: 특수 문자, 숫자 등을 제거하여 텍스트를 깔끔하게 만듭니다.
정규화: 어간 추출, 맞춤법 수정, 통일된 표현 사용 등을 통해 일관성 있는 형태로 변환합니다.
불용어 제거: 의미가 없는 단어를 제거합니다.
3. 특징 추출 (Feature Extraction)
텍스트 데이터를 수치적 형식으로 변환하여 모델이 학습할 수 있도록 합니다.
일반적으로 사용되는 방법에는 다음이 포함됩니다:
Bag of Words (BoW): 단어 빈도를 기반으로 표현.
TF-IDF (Term Frequency-Inverse Document Frequency): 단어의 중요도를 측정하여 가중치를 부여.
Word Embeddings: 단어를 고차원 공간에 벡터로 표현 (예: Word2Vec, GloVe).
4. 모델 선택 및 학습 (Model Selection and Training)
문제의 유형에 따라 적합한 모델을 선택하고, 수집한 데이터로 모델을 학습시킵니다.
분류, 회귀, 군집화 등 다양한 모델을 사용할 수 있습니다 (예: SVM, Random Forest, LSTM, BERT 등).
5. 모델 평가 (Model Evaluation)
학습한 모델의 성능을 평가합니다.
일반적으로 사용하는 평가 지표:
정확도 (Accuracy): 전체 예측 중 맞는 예측 비율.
정밀도 (Precision): 양성으로 예측한 것 중 실제 양성의 비율.
재현율 (Recall): 실제 양성 중 양성으로 예측한 비율.
F1 Score: 정밀도와 재현율의 조화 평균.
교차 검증을 통해 모델의 일반화 성능을 평가할 수 있습니다.
6. 하이퍼파라미터 튜닝 (Hyperparameter Tuning)
모델의 성능을 최적화하기 위해 하이퍼파라미터를 조정합니다.
Grid Search, Random Search, Bayesian Optimization 등 다양한 방법을 사용할 수 있습니다.
7. 모델 배포 (Model Deployment)
최종 모델을 실제 환경에 배포하여 사용합니다.
REST API 또는 웹 애플리케이션을 통해 사용자에게 제공할 수 있습니다.
8. 모델 모니터링 및 유지보수 (Model Monitoring and Maintenance)
배포된 모델의 성능을 지속적으로 모니터링하고, 필요할 경우 업데이트하거나 재학습시킵니다.

### 위의 순서에 따라 전처리하고 하이퍼파라미터 튜닝까지 코드 작성해줘

In [ ]:
pip install pandas numpy scikit-learn matplotlib seaborn konlpy hanspell


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt
from hanspell import spell_checker

# 1. 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 2. 불용어 로드
with open('stopwords_sorted.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 3. 텍스트 정규화 함수
okt = Okt()

def normalize_text(text):
    # 맞춤법 및 띄어쓰기 교정
    corrected_text = spell_checker.check(text).checked
    # 특수 문자 제거
    cleaned_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', corrected_text)
    # 토큰화 및 불용어 제거
    tokens = okt.nouns(cleaned_text)
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

# 4. 데이터 전처리
data['processed'] = data['판례내용'].apply(normalize_text)

# 5. 특징 추출
vectorizer = CountVectorizer()
word_count_matrix = vectorizer.fit_transform(data['processed'])

# 6. LDA 모델링 및 하이퍼파라미터 튜닝
lda = LDA(random_state=42)
param_grid = {
    'n_components': [5, 10, 15],
    'learning_decay': [0.5, 0.7, 0.9],
}

grid_search = GridSearchCV(lda, param_grid, cv=3)
grid_search.fit(word_count_matrix)

# 7. 최적 하이퍼파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# 8. 최적의 LDA 모델 사용
best_lda_model = grid_search.best_estimator_

# 9. 주제별 단어 시각화 함수
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(1, 5, figsize=(15, 6), sharex=True)
    axes = axes.flatten()

    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        sns.barplot(x=weights, y=top_features, ax=ax)
        ax.set_title(f'Topic {topic_idx + 1}', fontsize=12)
        ax.set_xlabel('Weight')
        ax.set_ylabel('Words')

    plt.suptitle(title, fontsize=16)
    plt.show()

# 10. 주제별 단어 시각화
plot_top_words(best_lda_model, vectorizer.get_feature_names_out(), 10, 'LDA Topic Words')


### 불용어사전 -> 형태소분석 -> LDA 주제 도출 -> 시각화

불용어사전 로드 및 판례내용 열에서 불용어 제거

불용어 사전을 불러오고, 텍스트 데이터에서 불용어를 제거합니다.
명사 추출 및 빈도 수 계산

KoNLPy의 Okt 형태소 분석기를 사용하여 명사를 추출하고, 이를 빈도 수로 계산합니다.
LDA 모델링

명사로 구성된 문서를 토큰화한 후, scikit-learn의 LDA 모델을 사용해 주제를 도출합니다.
주제별 단어 시각화

각 주제별로 가장 중요한 단어를 Seaborn과 matplotlib을 사용해 시각화합니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt

# 1. 불용어 로드 및 텍스트 전처리
with open('stopwords_sorted.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

data = pd.read_csv('판례.csv')  # 판례 데이터 로드
okt = Okt()

def preprocess_text(text):
    tokens = okt.nouns(text)  # 명사 추출
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return ' '.join(tokens)

data['processed'] = data['판례내용'].apply(preprocess_text)

# 2. 명사 추출 후 빈도수 계산
vectorizer = CountVectorizer()
word_count_matrix = vectorizer.fit_transform(data['processed'])
word_freq = np.array(word_count_matrix.sum(axis=0)).flatten()
word_names = vectorizer.get_feature_names_out()

word_freq_df = pd.DataFrame({'word': word_names, 'freq': word_freq}).sort_values(by='freq', ascending=False)
print(word_freq_df.head())

# 3. LDA 주제 도출
lda = LDA(n_components=5, random_state=42)  # 주제 개수는 5로 설정
lda.fit(word_count_matrix)

# 4. 주제별 단어 시각화
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(1, 5, figsize=(15, 6), sharex=True)
    axes = axes.flatten()
    
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]
        
        ax = axes[topic_idx]
        sns.barplot(x=weights, y=top_features, ax=ax)
        ax.set_title(f'Topic {topic_idx + 1}', fontsize=12)
        ax.set_xlabel('Weight')
        ax.set_ylabel('Words')
    
    plt.suptitle(title, fontsize=16)
    plt.show()

# 주제별 시각화
plot_top_words(lda, vectorizer.get_feature_names_out(), 10, 'LDA Topic Words')



로컬폴더에서 파일 불러올 때

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt
from hanspell import spell_checker
import os

# 1. 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 2. 불용어 로드 (로컬 폴더 경로 지정)
file_path = 'data/stopwords_sorted.txt'  # 경로를 맞춰주세요
with open(file_path, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 3. 텍스트 정규화 함수
okt = Okt()

def normalize_text(text):
    # 맞춤법 및 띄어쓰기 교정
    corrected_text = spell_checker.check(text).checked
    # 특수 문자 제거
    cleaned_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', corrected_text)
    # 토큰화 및 불용어 제거
    tokens = okt.nouns(cleaned_text)
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

# 4. 데이터 전처리
data['processed'] = data['판례내용'].apply(normalize_text)

# 5. 특징 추출
vectorizer = CountVectorizer()
word_count_matrix = vectorizer.fit_transform(data['processed'])

# 6. LDA 모델링 및 하이퍼파라미터 튜닝
lda = LDA(random_state=42)
param_grid = {
    'n_components': [5, 10, 15],
    'learning_decay': [0.5, 0.7, 0.9],
}

grid_search = GridSearchCV(lda, param_grid, cv=3)
grid_search.fit(word_count_matrix)

# 7. 최적 하이퍼파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# 8. 최적의 LDA 모델 사용
best_lda_model = grid_search.best_estimator_

# 9. 주제별 단어 시각화 함수
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(1, 5, figsize=(15, 6), sharex=True)
    axes = axes.flatten()

    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        sns.barplot(x=weights, y=top_features, ax=ax)
        ax.set_title(f'Topic {topic_idx + 1}', fontsize=12)
        ax.set_xlabel('Weight')
        ax.set_ylabel('Words')

    plt.suptitle(title, fontsize=16)
    plt.show()

# 10. 주제별 단어 시각화
plot_top_words(best_lda_model, vectorizer.get_feature_names_out(), 10, 'LDA Topic Words')


### 판례내용분석_기본

 * 주요 법률 용어 빈도 분석
    * TF-IDF (Term Frequency - Inverse Document Frequency) 분석
    * N-그램 분석
    * LDA (Latent Dirichlet Allocation)를 통한 주제 모델링
    * 단어간 상관관계 분석
    * 네트워크 분석 (단어-법률, 판사-변호사 관계)
    * 감정 분석 (법률 문서의 긍정/부정 감정 분류)

데이터 로드: service.csv에서 법률 문서 데이터를 불러옵니다.
불용어 로드: 로컬에서 불용어 파일을 불러옵니다.
텍스트 정규화: 문서 내용을 맞춤법 교정하고, 특수 문자를 제거한 후 불용어를 제거하여 명사를 추출합니다.
주요 법률 용어 빈도 분석: 가장 많이 사용된 법률 용어를 시각화합니다.
TF-IDF 분석: TF-IDF 행렬을 생성합니다.
N-그램 분석: 2-그램 또는 다른 N-그램을 분석하고 시각화합니다.
LDA 주제 모델링: LDA 모델을 학습하고 최적의 하이퍼파라미터를 찾아 주제를 도출합니다.
주제별 단어 시각화: 각 주제에서 중요한 단어를 시각화합니다.
단어 간 상관관계 분석: 특정 단어 간의 상관관계를 분석하여 시각화합니다.
네트워크 분석: 단어 간의 관계를 네트워크 형태로 시각화합니다.

In [ ]:
pip install pandas numpy scikit-learn matplotlib seaborn konlpy hanspell networkx nltk textblob


import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt
from hanspell import spell_checker
from nltk import ngrams
from collections import Counter
import networkx as nx
from textblob import TextBlob

# 1. 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 2. 불용어 로드
file_path = 'data/stopwords_sorted.txt'  # 경로를 맞춰주세요
with open(file_path, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 3. 텍스트 정규화 함수
okt = Okt()

def normalize_text(text):
    corrected_text = spell_checker.check(text).checked  # 맞춤법 및 띄어쓰기 교정
    cleaned_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', corrected_text)  # 특수 문자 제거
    tokens = okt.nouns(cleaned_text)  # 명사 추출
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return ' '.join(tokens)

# 4. 데이터 전처리
data['processed'] = data['판례내용'].apply(normalize_text)

# 5. 주요 법률 용어 빈도 분석
def plot_top_terms(data, num_terms=10):
    all_terms = ' '.join(data['processed'])
    term_counts = Counter(all_terms.split())
    most_common_terms = term_counts.most_common(num_terms)
    
    terms, counts = zip(*most_common_terms)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(terms))
    plt.title('Top Legal Terms Frequency')
    plt.xlabel('Frequency')
    plt.ylabel('Terms')
    plt.show()

plot_top_terms(data)

# 6. TF-IDF 분석
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# 7. N-그램 분석 (예: 2-그램)
def plot_ngrams(data, n=2, num_terms=10):
    all_terms = ' '.join(data['processed'])
    n_grams = ngrams(all_terms.split(), n)
    ngram_counts = Counter(n_grams)
    most_common_ngrams = ngram_counts.most_common(num_terms)
    
    ngrams_list = [' '.join(ngram) for ngram, _ in most_common_ngrams]
    counts = [count for _, count in most_common_ngrams]
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=counts, y=ngrams_list)
    plt.title(f'Top {n}-grams Frequency')
    plt.xlabel('Frequency')
    plt.ylabel(f'{n}-grams')
    plt.show()

plot_ngrams(data, n=2)

# 8. LDA 주제 모델링
lda_vectorizer = CountVectorizer()
word_count_matrix = lda_vectorizer.fit_transform(data['processed'])

lda = LDA(random_state=42)
param_grid = {
    'n_components': [5, 10, 15],
    'learning_decay': [0.5, 0.7, 0.9],
}

grid_search = GridSearchCV(lda, param_grid, cv=3)
grid_search.fit(word_count_matrix)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

best_lda_model = grid_search.best_estimator_

# 9. 주제별 단어 시각화
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(1, 5, figsize=(15, 6), sharex=True)
    axes = axes.flatten()

    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        sns.barplot(x=weights, y=top_features, ax=ax)
        ax.set_title(f'Topic {topic_idx + 1}', fontsize=12)
        ax.set_xlabel('Weight')
        ax.set_ylabel('Words')

    plt.suptitle(title, fontsize=16)
    plt.show()

plot_top_words(best_lda_model, lda_vectorizer.get_feature_names_out(), 10, 'LDA Topic Words')

# 10. 단어 간 상관관계 분석
def plot_word_correlation(data, num_terms=10):
    all_terms = ' '.join(data['processed'])
    term_counts = Counter(all_terms.split())
    common_terms = [term for term, count in term_counts.most_common(num_terms)]
    
    correlation_matrix = pd.DataFrame(index=common_terms, columns=common_terms).fillna(0)
    
    for term in common_terms:
        for other_term in common_terms:
            if term != other_term:
                correlation = all_terms.count(term) * all_terms.count(other_term)
                correlation_matrix.loc[term, other_term] = correlation

    sns.heatmap(correlation_matrix, annot=True, fmt=".0f", cmap='coolwarm')
    plt.title('Word Correlation Matrix')
    plt.show()

plot_word_correlation(data)

# 11. 네트워크 분석
def plot_word_network(data, num_terms=10):
    all_terms = ' '.join(data['processed'])
    term_counts = Counter(all_terms.split())
    common_terms = [term for term, count in term_counts.most_common(num_terms)]

    G = nx.Graph()

    for term in common_terms:
        G.add_node(term)

    for i, term1 in enumerate(common_terms):
        for term2 in common_terms[i + 1:]:
            weight = all_terms.count(term1) * all_terms.count(term2)
            if weight > 0:
                G.add_edge(term1, term2, weight=weight)

    pos = nx.spring_layout(G)
    plt.figure(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightblue')
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos, font_size=12)
    plt.title('Word Network')
    plt.show()

plot_word_network(data)

# 12. 감정 분석
def sentiment_analysis(data):
    data['sentiment'] = data['processed'].apply(lambda x: TextBlob(x).sentiment.polarity)
    data['sentiment_label'] = data['sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')
    
    sentiment_counts = data['sentiment_label'].value_counts()
    
    plt.figure(figsize=(8, 5))
    sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)
    plt.title('Sentiment Analysis of Legal Documents')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.show()

sentiment_analysis(data)


#### 전처리 순서 및 설정 순서

텍스트 전처리 과정에서는 토큰화, 정제, 정규화, 불용어 제거 등의 단계를 순차적으로 수행합니다. 각 단계의 역할과 순서를 설명드리겠습니다:
- 인코딩: 텍스트 데이터 로드 시 UTF-8 인코딩 사용
- 토큰화: 형태소 분석을 통해 텍스트를 단어 단위로 분리
- 정제: 불필요한 특수 문자나 공백 등을 제거
- 정규화: 의미가 동일한 단어들을 통일
- 불용어 제거: 사전에 정의된 불용어 리스트를 사용해 필요 없는 단어 제거

In [ ]:
data = pd.read_csv('filename.csv', encoding='utf-8')

okt = Okt()
tokens = okt.morphs(text)  # 형태소 분석 후 단어 추출

import re
cleaned_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]', ' ', text)

tokens = [word for word in tokens if word not in stopwords]


정규화는 텍스트를 일관되게 처리하여 데이터의 품질을 높이는 단계로, 일반적으로 어형(표현 방식)이 다른 단어들을 동일하게 맞추는 작업을 포함

1. 어간 추출 (Stemming)
단어의 변형된 형태(어미나 접미사)를 제거하고 기본 어간만을 남기는 방법입니다.

In [ ]:
from konlpy.tag import Okt
okt = Okt()
normalized_text = okt.normalize(text)


2. 어근 추출 (Lemmatization)
단어의 기본형(원형)으로 변환하는 방법입니다. 형태소 분석 단계에서 정규화 작업과 병행할 수 있습니다.

3. 맞춤법 및 띄어쓰기 교정
비정상적인 띄어쓰기나 맞춤법 오류를 수정하는 작업입니다. 이는 특히 소셜 미디어 텍스트에서 중요한데, 띄어쓰기나 맞춤법이 틀린 경우가 많기 때문입니다.

In [ ]:
from hanspell import spell_checker
corrected_text = spell_checker.check(text).checked


4. 통일된 표기 사용 (일반화)
여러 표현이 같은 의미를 가지는 경우, 이를 동일하게 정규화합니다. 예를 들어, "SNS"와 "소셜미디어" 같은 표현을 하나로 통일합니다.
사전 기반 정규화를 위해 직접 매핑 규칙을 만들거나, 특정 단어들을 변환하는 매핑 테이블을 작성할 수 있습니다.

In [ ]:
synonyms = {'SNS': '소셜미디어', '이메일': '전자우편'}
text = ' '.join([synonyms.get(word, word) for word in text.split()])


5. 단어 축약 및 통합 처리
소셜 미디어나 대화체 텍스트에서 자주 등장하는 축약어나 변형된 표현을 풀어서 표준 형태로 바꿉니다.
예: "ㅎㅎ" → "웃음", "ㅇㅇ" → "응"
축약어 리스트를 만들어 해당 단어들을 정규화할 수 있습니다.

6. 대소문자 변환 (영어 혼합 텍스트의 경우)
텍스트 내 대소문자를 일관되게 맞추는 작업입니다. 영어 혼합 텍스트가 포함된 경우 주로 사용하며, 모두 소문자로 변환하는 방식이 일반적입니다.

In [ ]:
text = text.lower()


7. 특정 패턴 변환 (숫자, 이모티콘 등)
숫자나 이모티콘, 날짜, 시간 등 특정 패턴을 가진 데이터도 정규화할 수 있습니다.

In [ ]:
text = re.sub(r'\d+', '<숫자>', text)  # 모든 숫자를 <숫자>로 변환


In [ ]:
import re
from konlpy.tag import Okt
from hanspell import spell_checker

okt = Okt()

def normalize_text(text):
    # 맞춤법 및 띄어쓰기 교정
    corrected_text = spell_checker.check(text).checked
    
    # 어간 추출 및 정규화
    normalized_text = okt.normalize(corrected_text)
    
    # 특수 문자 제거
    cleaned_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', normalized_text)
    
    # 축약어 및 통일된 표현 처리
    synonyms = {'SNS': '소셜미디어', '이메일': '전자우편'}
    final_text = ' '.join([synonyms.get(word, word) for word in cleaned_text.split()])
    
    return final_text

text = "SNS로 이메일을 보냈습니다 ㅎㅎ"
processed_text = normalize_text(text)
print(processed_text)
